<a href="https://colab.research.google.com/github/arunillo/6-lab-python/blob/main/6_%D0%BB%D0%B0%D0%B1%D0%B0_%D0%BF%D0%B8%D1%82%D0%BE%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#1 задания
import sqlite3
from datetime import datetime

def create_database():
    """
    Создает базу данных и таблицы для университета
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            # таблица students
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS students (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    group_name TEXT NOT NULL,
                    admission_year INTEGER NOT NULL,
                    average_grade REAL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            #  таблица courses
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS courses (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    course_name TEXT UNIQUE NOT NULL,
                    instructor TEXT NOT NULL,
                    credits INTEGER NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            print("База данных и таблицы успешно созданы!")

    except sqlite3.Error as e:
        print(f"Ошибка при создании базы данных: {e}")

if __name__ == "__main__":
    create_database()

База данных и таблицы успешно созданы!


In [9]:
#2 задания
import sqlite3
from typing import List, Dict, Optional

def get_connection():
    """Создает соединение с базой данных"""
    conn = sqlite3.connect('university.db')
    conn.row_factory = sqlite3.Row
    return conn

def add_student(first_name: str, last_name: str, group_name: str,
                admission_year: int, average_grade: float = None) -> Optional[int]:
    """
    Добавляет нового студента в базу данных
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade)
                VALUES (?, ?, ?, ?, ?)
            ''', (first_name, last_name, group_name, admission_year, average_grade))

            student_id = cursor.lastrowid
            print(f"Студент {first_name} {last_name} успешно добавлен с ID: {student_id}")
            return student_id

    except sqlite3.Error as e:
        print(f"Ошибка при добавлении студента: {e}")
        return None

def get_all_students() -> List[Dict]:
    """
    Возвращает список всех студентов
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT * FROM students ORDER BY last_name, first_name')
            students = [dict(row) for row in cursor.fetchall()]
            return students

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов: {e}")
        return []

def get_students_by_group(group_name: str) -> List[Dict]:
    """
    Возвращает студентов указанной группы
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT * FROM students
                WHERE group_name = ?
                ORDER BY last_name, first_name
            ''', (group_name,))
            students = [dict(row) for row in cursor.fetchall()]
            return students

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов группы: {e}")
        return []

def update_student_grade(student_id: int, new_grade: float) -> bool:
    """
    Обновляет средний балл студента
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                UPDATE students
                SET average_grade = ?
                WHERE id = ?
            ''', (new_grade, student_id))

            if cursor.rowcount > 0:
                print(f"Оценка студента с ID {student_id} успешно обновлена")
                return True
            else:
                print(f"Студент с ID {student_id} не найден")
                return False

    except sqlite3.Error as e:
        print(f"Ошибка при обновлении оценки: {e}")
        return False

def delete_student(student_id: int) -> bool:
    """
    Удаляет студента по ID
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('DELETE FROM students WHERE id = ?', (student_id,))

            if cursor.rowcount > 0:
                print(f"Студент с ID {student_id} успешно удален")
                return True
            else:
                print(f"Студент с ID {student_id} не найден")
                return False

    except sqlite3.Error as e:
        print(f"Ошибка при удалении студента: {e}")
        return False

# Тестирование функций
def test_student_operations():
    """Тестирование CRUD-операций для студентов"""
    print("\n=== Тестирование операций со студентами ===")

    # Добавление студентов
    student1_id = add_student("Аружан", "Мухаметжанова", "ГР-01", 2023, 4.5)
    student2_id = add_student("Мария", "Иванова", "ГР-01", 2023, 4.8)
    student3_id = add_student("Алексей", "Сидоров", "ГР-02", 2023, 3.9)

    # Получение всех студентов
    all_students = get_all_students()
    print("\nВсе студенты:")
    for student in all_students:
        print(f"  {student['id']}: {student['first_name']} {student['last_name']} - {student['group_name']}")

    # Получение студентов группы
    group_students = get_students_by_group("ГР-01")
    print("\nСтуденты ГР-01:")
    for student in group_students:
        print(f"  {student['first_name']} {student['last_name']}")

    # Обновление оценки
    if student1_id:
        update_student_grade(student1_id, 4.9)

    # Удаление студента
    if student2_id:
        delete_student(student2_id)

if __name__ == "__main__":
    create_database()
    test_student_operations()

База данных и таблицы успешно созданы!

=== Тестирование операций со студентами ===
Студент Аружан Мухаметжанова успешно добавлен с ID: 11
Студент Мария Иванова успешно добавлен с ID: 12
Студент Алексей Сидоров успешно добавлен с ID: 13

Все студенты:
  12: Мария Иванова - ГР-01
  10: Ксения Куценко - ГР-01
  7: Аружан Мухаметжанова - ГР-01
  11: Аружан Мухаметжанова - ГР-01
  1: Иван Петров - ГР-03
  4: Иван Петров - ГР-01
  3: Алексей Сидоров - ГР-02
  6: Алексей Сидоров - ГР-02
  9: Алексей Сидоров - ГР-02
  13: Алексей Сидоров - ГР-02

Студенты ГР-01:
  Мария Иванова
  Ксения Куценко
  Аружан Мухаметжанова
  Аружан Мухаметжанова
  Иван Петров
Оценка студента с ID 11 успешно обновлена
Студент с ID 12 успешно удален


In [10]:
#3 задания
def create_relations_tables():
    """
    Создает таблицу для связей студентов и курсов
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            # Создание таблицы связей
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS student_courses (
                    student_id INTEGER,
                    course_id INTEGER,
                    enrolled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (student_id, course_id),
                    FOREIGN KEY (student_id) REFERENCES students(id) ON DELETE CASCADE,
                    FOREIGN KEY (course_id) REFERENCES courses(id) ON DELETE CASCADE
                )
            ''')

            # Включаем поддержку внешних ключей
            cursor.execute('PRAGMA foreign_keys = ON')

            print("Таблица связей успешно создана!")

    except sqlite3.Error as e:
        print(f"Ошибка при создании таблицы связей: {e}")

def add_course(course_name: str, instructor: str, credits: int) -> Optional[int]:
    """
    Добавляет новый курс
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO courses (course_name, instructor, credits)
                VALUES (?, ?, ?)
            ''', (course_name, instructor, credits))

            course_id = cursor.lastrowid
            print(f"Курс '{course_name}' успешно добавлен с ID: {course_id}")
            return course_id

    except sqlite3.IntegrityError:
        print(f"Курс '{course_name}' уже существует")
        return None
    except sqlite3.Error as e:
        print(f"Ошибка при добавлении курса: {e}")
        return None

def enroll_student_in_course(student_id: int, course_id: int) -> bool:
    """
    Записывает студента на курс
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT OR IGNORE INTO student_courses (student_id, course_id)
                VALUES (?, ?)
            ''', (student_id, course_id))

            if cursor.rowcount > 0:
                print(f"Студент {student_id} успешно записан на курс {course_id}")
                return True
            else:
                print(f"Студент {student_id} уже записан на курс {course_id}")
                return False

    except sqlite3.Error as e:
        print(f"Ошибка при записи на курс: {e}")
        return False

def get_student_courses(student_id: int) -> List[Dict]:
    """
    Возвращает курсы студента
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT c.* FROM courses c
                JOIN student_courses sc ON c.id = sc.course_id
                WHERE sc.student_id = ?
            ''', (student_id,))

            courses = [dict(row) for row in cursor.fetchall()]
            return courses

    except sqlite3.Error as e:
        print(f"Ошибка при получении курсов студента: {e}")
        return []

def get_course_students(course_id: int) -> List[Dict]:
    """
    Возвращает студентов курса
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT s.* FROM students s
                JOIN student_courses sc ON s.id = sc.student_id
                WHERE sc.course_id = ?
            ''', (course_id,))

            students = [dict(row) for row in cursor.fetchall()]
            return students

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов курса: {e}")
        return []

def transfer_student(student_id: int, new_group: str) -> bool:
    """
    Переводит студента в другую группу
    """
    try:
        with get_connection() as conn:
            cursor = conn.cursor()

            # Начинаем транзакцию
            cursor.execute('BEGIN TRANSACTION')

            try:
                # Обновляем группу студента
                cursor.execute('''
                    UPDATE students
                    SET group_name = ?
                    WHERE id = ?
                ''', (new_group, student_id))

                if cursor.rowcount == 0:
                    print(f"Студент с ID {student_id} не найден")
                    conn.rollback()
                    return False

                print(f"Студент {student_id} успешно переведен в группу {new_group}")
                conn.commit()
                return True

            except sqlite3.Error:
                conn.rollback()
                raise

    except sqlite3.Error as e:
        print(f"Ошибка при переводе студента: {e}")
        return False

def test_course_operations():
    """Тестирование операций с курсами"""
    print("\n=== Тестирование операций с курсами ===")

    # Создаем таблицу связей
    create_relations_tables()

    # Добавляем курсы
    math_id = add_course("Математика", "Проф. Иванов", 5)
    physics_id = add_course("Физика", "Проф. Петрова", 4)
    programming_id = add_course("Программирование", "Доц. Сидоров", 6)

    # Зачисление студентов на курсы
    enroll_student_in_course(1, math_id)
    enroll_student_in_course(1, programming_id)
    enroll_student_in_course(3, physics_id)

    # Получение курсов студента
    ivan_courses = get_student_courses(1)
    print(f"\nКурсы студента 1:")
    for course in ivan_courses:
        print(f"  {course['course_name']} - {course['instructor']}")

    # Перевод студента
    transfer_student(1, "ГР-03")

if __name__ == "__main__":
    test_course_operations()


=== Тестирование операций с курсами ===
Таблица связей успешно создана!
Курс 'Математика' уже существует
Курс 'Физика' уже существует
Курс 'Программирование' уже существует
Студент 1 успешно записан на курс None
Студент 1 успешно записан на курс None
Студент 3 успешно записан на курс None

Курсы студента 1:
  Математика - Проф. Иванов
  Программирование - Доц. Сидоров
Студент 1 успешно переведен в группу ГР-03


In [11]:
#4 задания
class UniversityDB:
    """
    Класс-обертка для работы с базой данных университета
    """
    def __init__(self, db_path: str = 'university.db'):
        self.db_path = db_path
        self.connection = None
        self.cursor = None

    def __enter__(self):
        self.connection = sqlite3.connect(self.db_path)
        self.connection.row_factory = sqlite3.Row
        self.cursor = self.connection.cursor()
        # Включаем поддержку внешних ключей
        self.cursor.execute('PRAGMA foreign_keys = ON')
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.connection:
            if exc_type is None:
                self.connection.commit()
            else:
                self.connection.rollback()
                print(f"Произошла ошибка: {exc_val}")
            self.connection.close()

    def execute_query(self, query: str, params: tuple = None) -> bool:
        """
        Выполняет SQL-запрос
        """
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            return True
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return False

    def fetch_all(self, query: str, params: tuple = None) -> List[Dict]:
        """
        Выполняет запрос и возвращает все результаты
        """
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            return [dict(row) for row in self.cursor.fetchall()]
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return []

    def fetch_one(self, query: str, params: tuple = None) -> Optional[Dict]:
        """
        Выполняет запрос и возвращает одну запись
        """
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            result = self.cursor.fetchone()
            return dict(result) if result else None
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return None

    def get_student_statistics(self) -> Dict:
        """
        Возвращает статистику по студентам
        """
        stats = {}

        # Общее количество студентов
        result = self.fetch_one("SELECT COUNT(*) as total FROM students")
        stats['total_students'] = result['total'] if result else 0

        # Средний балл по университету
        result = self.fetch_one("SELECT AVG(average_grade) as avg_grade FROM students WHERE average_grade IS NOT NULL")
        stats['university_avg_grade'] = round(result['avg_grade'], 2) if result and result['avg_grade'] else 0

        # Количество студентов по группам
        groups_stats = self.fetch_all('''
            SELECT group_name, COUNT(*) as count,
                   AVG(average_grade) as avg_grade
            FROM students
            GROUP BY group_name
        ''')
        stats['groups'] = groups_stats

        return stats

    def get_top_students(self, limit: int = 5) -> List[Dict]:
        """
        Возвращает топ N студентов по среднему баллу
        """
        return self.fetch_all('''
            SELECT first_name, last_name, group_name, average_grade
            FROM students
            WHERE average_grade IS NOT NULL
            ORDER BY average_grade DESC
            LIMIT ?
        ''', (limit,))

    def get_group_statistics(self) -> List[Dict]:
        """
        Возвращает статистику по группам
        """
        return self.fetch_all('''
            SELECT
                group_name,
                COUNT(*) as student_count,
                AVG(average_grade) as avg_grade,
                MIN(admission_year) as first_year,
                MAX(admission_year) as last_year
            FROM students
            GROUP BY group_name
            ORDER BY group_name
        ''')

    def search_students_by_name(self, name_part: str) -> List[Dict]:
        """
        Поиск студентов по части имени или фамилии
        """
        search_pattern = f'%{name_part}%'
        return self.fetch_all('''
            SELECT * FROM students
            WHERE first_name LIKE ? OR last_name LIKE ?
            ORDER BY last_name, first_name
        ''', (search_pattern, search_pattern))

def test_university_db():
    """Тестирование класса UniversityDB"""
    print("\n=== Тестирование UniversityDB ===")

    with UniversityDB() as db:
        # Статистика студентов
        stats = db.get_student_statistics()
        print(f"Общее количество студентов: {stats['total_students']}")
        print(f"Средний балл по университету: {stats['university_avg_grade']}")

        # Топ студентов
        top_students = db.get_top_students(3)
        print("\nТоп 3 студентов:")
        for student in top_students:
            print(f"  {student['first_name']} {student['last_name']} - {student['average_grade']}")

        # Статистика по группам
        group_stats = db.get_group_statistics()
        print("\nСтатистика по группам:")
        for group in group_stats:
            print(f"  {group['group_name']}: {group['student_count']} студентов, средний балл: {group['avg_grade']}")

if __name__ == "__main__":
    test_university_db()


=== Тестирование UniversityDB ===
Общее количество студентов: 9
Средний балл по университету: 4.46

Топ 3 студентов:
  Иван Петров - 4.9
  Иван Петров - 4.9
  Аружан Мухаметжанова - 4.9

Статистика по группам:
  ГР-01: 4 студентов, средний балл: 4.9
  ГР-02: 4 студентов, средний балл: 3.9
  ГР-03: 1 студентов, средний балл: 4.9


In [ ]:
#5 задания
def display_students_table(students: List[Dict]):
    """
    Красиво отображает список студентов в виде таблицы
    """
    if not students:
        print("Студенты не найдены")
        return

    # Заголовок таблицы
    print("\n" + "="*80)
    print(f"{'ID':<3} {'Фамилия':<15} {'Имя':<15} {'Группа':<8} {'Год':<6} {'Ср. балл':<8}")
    print("-"*80)

    # Данные
    for student in students:
        student_id = student.get('id', '')
        last_name = student.get('last_name', '')[:14]
        first_name = student.get('first_name', '')[:14]
        group_name = student.get('group_name', '')[:7]
        admission_year = student.get('admission_year', '')
        average_grade = student.get('average_grade', '')

        print(f"{student_id:<3} {last_name:<15} {first_name:<15} {group_name:<8} {admission_year:<6} {average_grade:<8}")

    print("="*80)

def add_student_interactive():
    """Интерактивное добавление студента"""
    print("\n--- Добавление студента ---")

    first_name = input("Имя: ").strip()
    last_name = input("Фамилия: ").strip()
    group_name = input("Группа: ").strip()

    # Валидация года поступления
    while True:
        try:
            admission_year = int(input("Год поступления: "))
            if 2000 <= admission_year <= 2030:
                break
            else:
                print("Год должен быть между 2000 и 2030")
        except ValueError:
            print("Введите корректный год")

    # Валидация среднего балла
    average_grade = None
    grade_input = input("Средний балл (Enter чтобы пропустить): ").strip()
    if grade_input:
        try:
            average_grade = float(grade_input)
            if not (0 <= average_grade <= 5):
                print("Балл должен быть между 0 и 5")
                average_grade = None
        except ValueError:
            print("Введите корректное число")

    # Добавление в базу данных
    with UniversityDB() as db:
        success = db.execute_query('''
            INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade)
            VALUES (?, ?, ?, ?, ?)
        ''', (first_name, last_name, group_name, admission_year, average_grade))

        if success:
            print("Студент успешно добавлен!")
        else:
            print("Ошибка при добавлении студента")

def display_all_students():
    """Отображение всех студентов"""
    with UniversityDB() as db:
        students = db.fetch_all('SELECT * FROM students ORDER BY last_name, first_name')
        display_students_table(students)

def search_student_by_last_name():
    """Поиск студента по фамилии"""
    print("\n--- Поиск студента по фамилии ---")
    last_name_part = input("Введите часть фамилии: ").strip()

    with UniversityDB() as db:
        students = db.search_students_by_name(last_name_part)
        if students:
            display_students_table(students)
        else:
            print("Студенты не найдены")

def update_student_grade_interactive():
    """Обновление оценки студента"""
    print("\n--- Обновление оценки студента ---")

    try:
        student_id = int(input("ID студента: "))
        new_grade = float(input("Новый средний балл: "))

        if not (0 <= new_grade <= 5):
            print("Балл должен быть между 0 и 5")
            return

        with UniversityDB() as db:
            success = db.execute_query('''
                UPDATE students SET average_grade = ? WHERE id = ?
            ''', (new_grade, student_id))

            if success and db.cursor.rowcount > 0:
                print("Оценка успешно обновлена!")
            else:
                print("Студент не найден или произошла ошибка")

    except ValueError:
        print("Введите корректные данные")

def delete_student_interactive():
    """Удаление студента"""
    print("\n--- Удаление студента ---")

    try:
        student_id = int(input("ID студента для удаления: "))

        with UniversityDB() as db:
            # Сначала проверим существование студента
            student = db.fetch_one('SELECT * FROM students WHERE id = ?', (student_id,))

            if student:
                print(f"Найден студент: {student['first_name']} {student['last_name']}")
                confirm = input("Вы уверены, что хотите удалить? (y/N): ").strip().lower()

                if confirm == 'y':
                    success = db.execute_query('DELETE FROM students WHERE id = ?', (student_id,))
                    if success:
                        print("Студент успешно удален!")
                else:
                    print("Удаление отменено")
            else:
                print("Студент не найден")

    except ValueError:
        print("Введите корректный ID")

def display_statistics():
    """Отображение статистики"""
    with UniversityDB() as db:
        stats = db.get_student_statistics()

        print("\n=== СТАТИСТИКА УНИВЕРСИТЕТА ===")
        print(f"Общее количество студентов: {stats['total_students']}")
        print(f"Средний балл по университету: {stats['university_avg_grade']}")

        print("\n--- Статистика по группам ---")
        for group in stats['groups']:
            print(f"Группа {group['group_name']}: {group['count']} студентов, средний балл: {group['avg_grade'] or 'N/A'}")

        # Топ студентов
        top_students = db.get_top_students(5)
        if top_students:
            print("\n--- Топ-5 студентов ---")
            for i, student in enumerate(top_students, 1):
                print(f"{i}. {student['first_name']} {student['last_name']} ({student['group_name']}) - {student['average_grade']}")

def main():
    """
    Главная функция приложения
    """
    print("Добро пожаловать в систему учета студентов университета!")

    while True:
        print("\n" + "="*50)
        print("           УНИВЕРСИТЕТСКИЙ УЧЕТ")
        print("="*50)
        print("1. Добавить студента")
        print("2. Просмотреть всех студентов")
        print("3. Найти студента по фамилии")
        print("4. Обновить оценку студента")
        print("5. Удалить студента")
        print("6. Показать статистику")
        print("7. Выход")
        print("-"*50)

        choice = input("Выберите действие (1-7): ").strip()

        if choice == '1':
            add_student_interactive()
        elif choice == '2':
            display_all_students()
        elif choice == '3':
            search_student_by_last_name()
        elif choice == '4':
            update_student_grade_interactive()
        elif choice == '5':
            delete_student_interactive()
        elif choice == '6':
            display_statistics()
        elif choice == '7':
            print("До свидания!")
            break
        else:
            print("Неверный выбор! Пожалуйста, выберите от 1 до 7")

        input("\nНажмите Enter для продолжения...")

if __name__ == "__main__":
    # Создаем базу данных и таблицы при первом запуске
    create_database()
    create_relations_tables()

    # Запускаем главное меню
    main()

База данных и таблицы успешно созданы!
Таблица связей успешно создана!
Добро пожаловать в систему учета студентов университета!

           УНИВЕРСИТЕТСКИЙ УЧЕТ
1. Добавить студента
2. Просмотреть всех студентов
3. Найти студента по фамилии
4. Обновить оценку студента
5. Удалить студента
6. Показать статистику
7. Выход
--------------------------------------------------
